# 複数のLLMモデルをLangchainで切り替えて使用するサンプル

このノートブックでは以下のモデルを切り替えて使用できます：
- OpenAI: GPT-4o-mini, GPT-4o
- Anthropic: Claude Opus 4 thinking
- Google: Gemini, Gemini Pro
- DeepSeek: DeepSeek R1 0528
- Groq: Grok 3 mini reasoning(high)

In [ ]:
# 必要なライブラリのインストール
!pip install langchain langchain_openai langchain_anthropic langchain_google_genai langchain_groq

In [ ]:
# APIキーの設定
import os
from google.colab import userdata

# Google Colabシークレットからキーを取得
# あらかじめ「シークレット」タブで各APIキーを登録しておく必要があります
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

# 直接入力する場合（非推奨）
# os.environ["OPENAI_API_KEY"] = "your-api-key"

In [ ]:
# 必要なインポート
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage, SystemMessage
import requests

In [ ]:
# DeepSeekのAPI呼び出し用の関数（Langchainでは現在直接サポートされていないため）
def call_deepseek_api(prompt, model="deepseek-chat-r1-0528"):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.environ['DEEPSEEK_API_KEY']}"
    }
    
    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}]
    }
    
    response = requests.post(
        "https://api.deepseek.ai/v1/chat/completions",
        json=data,
        headers=headers
    )
    
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.status_code} - {response.text}"

In [ ]:
# 各モデルを取得する関数
def get_model(model_name):
    """
    指定されたモデル名に基づいてLangChainモデルインスタンスを返す
    """
    if model_name == "gpt-4o-mini":
        return ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    
    elif model_name == "gpt-4o":
        return ChatOpenAI(model="gpt-4o", temperature=0.7)
        
    elif model_name == "claude-opus-4-thinking":
        return ChatAnthropic(model="claude-3-opus-20240229", temperature=0.7)
        
    elif model_name == "gemini":
        return ChatGoogleGenerativeAI(model="gemini-1.0-pro", temperature=0.7)
        
    elif model_name == "gemini-pro":
        return ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.7)
        
    elif model_name == "grok-3-mini-reasoning":
        return ChatGroq(model="grok-3-mini", temperature=0.7)
        
    else:
        raise ValueError(f"Unknown model: {model_name}")

In [ ]:
# チャットを実行する関数
def chat_with_model(model_name, prompt, system_message=None):
    """
    指定されたモデルでチャットを実行する
    
    Args:
        model_name (str): 使用するモデル名
        prompt (str): プロンプト
        system_message (str): システムメッセージ（オプション）
    """
    # DeepSeek R1 0528は別処理
    if model_name == "deepseek-r1-0528":
        return call_deepseek_api(prompt)
    
    model = get_model(model_name)
    messages = []
    
    # システムメッセージの設定（指定がある場合）
    if system_message:
        if model_name == "claude-opus-4-thinking":
            messages.append(SystemMessage(content=system_message + "\nUse thinking out loud to reason through this problem step by step."))
        elif model_name == "grok-3-mini-reasoning":
            messages.append(SystemMessage(content=system_message + "\nUse step by step reasoning to solve this problem."))
        else:
            messages.append(SystemMessage(content=system_message))
    elif model_name == "claude-opus-4-thinking":
        messages.append(SystemMessage(content="Use thinking out loud to reason through this problem step by step."))
    elif model_name == "grok-3-mini-reasoning":
        messages.append(SystemMessage(content="Use step by step reasoning to solve this problem."))
    
    # ユーザーメッセージの追加
    messages.append(HumanMessage(content=prompt))
    
    # モデルの呼び出し
    response = model.invoke(messages)
    return response.content

## モデルのテスト

In [ ]:
# 使用可能なモデル一覧
models = [
    "gpt-4o-mini",
    "gpt-4o",
    "claude-opus-4-thinking", 
    "gemini",
    "gemini-pro",
    "deepseek-r1-0528",
    "grok-3-mini-reasoning"
]

# モデルを選択するためのドロップダウンを作成
from ipywidgets import interact, Dropdown

@interact(model_name=Dropdown(options=models, value="gpt-4o-mini"))
def test_model(model_name):
    print(f"選択されたモデル: {model_name}")

In [ ]:
# 選んだモデルで質問を試す
def ask_question(model_name, prompt, system_message=None):
    print(f"モデル {model_name} に質問中...")
    try:
        response = chat_with_model(model_name, prompt, system_message)
        print("\n回答:")
        print(response)
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")
        
# 質問例
sample_prompt = "保険商品のプライシングについて、リスクベースのアプローチを簡単に説明してください。"

In [ ]:
# 選択したモデルで質問を試す
model_name = "gpt-4o-mini"  # ここでモデル名を変更してテスト
ask_question(model_name, sample_prompt)

## インタラクティブなクエリ機能

In [ ]:
from ipywidgets import interact, Text, Textarea

@interact(model_name=Dropdown(options=models, value="gpt-4o-mini"),
          prompt=Textarea(value=sample_prompt, description="質問:", layout={'width': '100%', 'height': '100px'}),
          system_message=Text(value="", description="システムメッセージ(オプション):", layout={'width': '100%'}))
def interactive_chat(model_name, prompt, system_message=""):
    if not prompt.strip():
        print("質問を入力してください")
        return
    
    system_message = system_message if system_message.strip() else None
    ask_question(model_name, prompt, system_message)